In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [2]:
class socketio(object):
    @classmethod
    def emit(msg, msg1, msg2, **kw):
        print(msg1, msg2)

In [3]:
market = {'name': 'Eyelash', 'directory': "E:\\glitter\\Eyelashes"}
lid = 'eyelash@glittergroupcn.com'
lpwd = 'Wangzhen68970747'
p4p = P4P(market, lid, lpwd, socketio)

E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [ ]:
def monitor(self, group='all', tid=None, socketio=None, tasks=None):
    if tid:
        tasks[tid]['is_running'] = True
        msg = {'name': 'P4P.monitor', 'tid': tid, 'group': group, 'is_last_run': tasks[tid]['is_last_run']}
        socketio.emit('event_task_start_running', msg, namespace='/markets', broadcast=True)

    try:
        print('Task Monitor start to run with group="'+group+'"')
        keywords = []
        self.load_keywords('monitor')
        if tid:
            tasks[tid]['progress'] = 1
            socketio.emit('event_task_progress', {'tid': tid, 'progress': 1}, namespace='/markets', broadcast=True)
        with self.lock:
            self.load_url()
            if tid:
                tasks[tid]['progress'] = 2
                socketio.emit('event_task_progress', {'tid': tid, 'progress': 2}, namespace='/markets', broadcast=True)
            all_kws_count = int(self.browser.find_element_by_css_selector('a.all-kwcount span').text)

            kws_count = 0
            while True:
                table_reloaded = True
                css_selector = "div.keyword-manage .bp-table-main-wraper>table"
                table = self.browser.find_element_by_css_selector(css_selector)
                print(self.keywords_list['monitor'])
                idx = 0
                while True:
                    kws_count += 1
                    if tid:
                        tasks[tid]['progress'] = int(kws_count/all_kws_count*97)+3
                        socketio.emit('event_task_progress', {'tid': tid, 'progress': tasks[tid]['progress']}, namespace='/markets', broadcast=True)

                    print(table_reloaded, end=" > ")
                    if table_reloaded:
                        trs =  table.find_elements_by_css_selector('tbody tr')
                        table_reloaded = False

                    print('index:', idx, len(trs), end=' > ')
                    if idx >= len(trs):
                        print('')
                        break

                    try:
                        tr = trs[idx]
                        ActionChains(self.browser).move_to_element(tr).perform()
                        id = tr.find_element_by_css_selector('td:first-child input').get_attribute('value').strip()
                        if id not in self.keywords_list['monitor']:
                            print('skipped_not_in_monitoring')
                            idx += 1
                            continue

                        grp = tr.find_element_by_css_selector('td:nth-child(4)').text.strip()
                        if group != 'all':
                            if group != grp:
                                print('skipped_not_in_group', group, grp)
                                idx += 1
                                continue

                        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()

                        print(kws, end=' > ')

                        if not self.open_price_dialog(tr):
                            print('skipped_2')
                            idx += 1
                            continue

                        [prices, sponsors] = self.find_prices_and_sponsors(close=False)

                        current_position = self.find_sponsor_list_position(sponsors=sponsors['sponsor_list'])
                        
                        if current_position == -1 and prices:
                            keywords.append([dt, id, kws, grp, prices, sponsors])
                        
                        click_position = self.get_click_position(id, current_position)
                        if click_position != -1:
                            self.set_price(position=click_position)
                        else:
                            webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
                        if not self.is_on(tr):
                            print('turn_on', end=" > ")
                            self.turn_on(tr)
                            table_reloaded = True

                        idx += 1
                        print(' >>>>>> successful end ')
                    except StaleElementReferenceException as e:
                        table_reloaded = True
                        print(' >>>>>> failed ... .... , retry .... ...')
                        continue

                if not self.next_page():
                    break

        if keywords:
            self.save_crawling_result(keywords)
        if tid:
            tasks[tid]['progress'] = 100
            socketio.emit('event_task_progress', {'tid': tid, 'progress': 100}, namespace='/markets', broadcast=True)
    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()
    finally:
        if tid:
            tasks[tid]['is_running'] = False
            tasks[tid]['progress'] = 0
            if tasks[tid]['is_last_run']:
                del tasks[tid]
                socketio.emit('event_task_last_run_finished', {'tid': tid}, namespace='/markets', broadcast=True)

p4p.monitor = types.MethodType(monitor, p4p)

In [ ]:
def get_click_position(self, kws_id, current_pos):
    now = pendulum.now()
    if kws_id not in self.kws_tracking:
        tracking = {}
        tracking['position'] = [self.default_position, now, False]
        self.kws_tracking[kws_id] = tracking
        return self.default_position
    else:
        tracking = self.kws_tracking[kws_id]
        [pos, dt, is_out] = tracking['position']
        if current_pos == -1:
            pos = 3
            tracking['position'] = [pos, now, True]
            return pos
        elif is_out:
            period = now.diff(dt).in_minutes()
            if period <= 2:
                if current_pos >= 3:
                    return -1
                else:
                    pos = 3
                    tracking['position'] = [pos, dt, True]
            elif period > 2 and period <=5:
                if current_pos >= 4:
                    return -1
                else:
                    pos = 4
                    tracking['position'] = [pos, dt, True]
            else:
                pos = 5
                tracking['position'] = [pos, dt, False]
            return pos
        elif current_pos == self.default_position:
            return -1
        else:
            return self.default_position
            #calculate the escaped time since in this position
        
p4p.get_click_position = types.MethodType(get_click_position, p4p)

In [ ]:
def save_crawling_result(self, keywords):
    root = self.market['directory'] + '_config'
    date_str = keywords[0][0].split(' ')[0]
    fn = 'p4p_keywords_crawl_result_'+date_str+'.json.gz'
    print(fn)
    JSON.serialize(keywords, root, [], fn, append=True)
p4p.save_crawling_result = types.MethodType(save_crawling_result, p4p)

In [ ]:
def set_price(self, tr=None, position=None, price=None):
    if not position and not price:
        return

    # pos = 3
    max_price = 30
    # sum = 0
    # id = tr.find_element_by_css_selector('td:first-child input').get_attribute('value').strip()
    # for prices in self.recent_prices[id]:
    #     sum += float(prices[pos])
    # mean_price = sum/len(self.recent_prices[id])
    # if max_price > (mean_price + 0.1):
    #     max_price = (mean_price + 0.1)

    if tr and not self.open_price_dialog(tr):
        return False

    success = True
    WebDriverWait(self.browser, 15).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.bp-loading-panel')))
    while True:
        try:
            price_table_tbody_selector = ".sc-manage-edit-price-dialog table tbody,.sc-manage-edit-price-dialog p.util-clearfix"
            price_table_tbody = WebDriverWait(self.browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, price_table_tbody_selector)))

            if price_table_tbody.tag_name == 'p':
                success = False
                break

            if price:
                css_selector = '.sc-manage-edit-price-dialog span[data-role="span-baseprice"'
                min_price = float(self.browser.find_element_by_css_selector(css_selector).text)
                if price < min_price:
                    price = min_price

                js_templ = "document.querySelector('{selector}').value = '{value}';"
                css_selector = '.sc-manage-edit-price-dialog input[name="addPrice"]'
                js = js_templ.format(selector=css_selector, value=price)
                self.browser.execute_script(js)
            else:
                while True:
                    btn = price_table_tbody.find_element_by_css_selector('td:nth-child('+str(int(position)+1)+') a, td:last-child a')
                    p = btn.text.strip()
                    cp = self.browser.find_element_by_css_selector('.sc-manage-edit-price-dialog input[name="addPrice"]').get_attribute('value')

                    if cp == p:
                        webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
                        print('不需要改变价格', end=' > ')
                        return success
                    if float(p) <= max_price:
                        self.click(btn)
                        break
                    if position >= 5:
                        break
                    position += 1

            confirm = self.browser.find_element_by_css_selector('.ui2-dialog-btn input[data-role="confirm"]')
            self.click(confirm)
            break
        except StaleElementReferenceException:
            self.browser.implicitly_wait(0.5)
            continue
    return success
p4p.set_price = types.MethodType(set_price, p4p)

In [ ]:
p4p.default_position = 5
p4p.kws_tracking = {}

In [4]:
p4p.monitor()

Task Monitor start to run with group="all"
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!
primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy
{'77942874823': 'strip eyelash', '77942874901': 'synthetic eyelashes'}
True > index: 0 50 > skipped_not_in_monitoring
False > index: 1 50 > skipped_not_in_monitoring
False > index: 2 50 > skipped_not_in_monitoring
False > index: 3 50 > skipped_not_in_monitoring
False > index: 4 50 > skipped_not_in_monitoring
False > index: 5 50 > skipped_not_in_monitoring
False > index: 6 50 > skipped_not_in_monitoring
False > index: 7 50 > skipped_not_in_monitoring
False > index: 8 50 > skipped_not_in_monitoring
False > index: 9 50 > skipped_not_in_monitoring
False > index: 10 50 > skipped_not_in_monitoring
False > index: 11 50 > skipped_not_in_monitoring
False > index: 12 50 > skipped_not_in_monitoring
False >

In [5]:
p4p.kws_tracking

{'77942874823': {'position': [5,
   DateTime(2018, 7, 14, 12, 8, 47, 349351, tzinfo=Timezone('Asia/Shanghai')),
   False]},
 '77942874901': {'position': [5,
   DateTime(2018, 7, 14, 12, 8, 49, 612284, tzinfo=Timezone('Asia/Shanghai')),
   False]}}

In [6]:
p4p.turn_all_off()

next_page: True
next_page: False


In [ ]:
# p4p.load_url()

# css_selector = "div.keyword-manage .bp-table-main-wraper>table"
# table = p4p.browser.find_element_by_css_selector(css_selector)
# trs = table.find_elements_by_css_selector('tbody tr')
# idx = 1
# for tr in trs:
#     print()
#     print(idx, end=': ')
#     idx += 1
    
#     kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
#     print(kws, end=": ")
        
#     if not p4p.open_price_dialog(tr):
#         print('skipped_2')
#         idx += 1
#         continue
        
#     [prices, sponsors] = p4p.find_prices_and_sponsors()
#     print(prices, sponsors)
#     if idx > 10:
#         break

In [ ]:
tr = trs[8]
kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
print(kws, end=": ")

if not p4p.open_price_dialog(tr):
    print('skipped_2')
else:
#     time.sleep(1)
    prices = p4p.find_prices()

In [ ]:
p4p.find_sponsor_list_position(kws='strip eyelash')

In [ ]:
import pendulum

In [ ]:
pendulum.now()